In [1]:
%scala
import org.apache.spark.streaming.{Seconds, StreamingContext}
import org.apache.spark.SparkContext._
import org.apache.spark.streaming.twitter
import org.apache.spark.SparkConf
import org.apache.spark.streaming._
import org.apache.spark.{SparkContext, SparkConf}
import org.apache.spark.storage.StorageLevel
import twitter4j._
import twitter4j.auth.Authorization
import twitter4j.conf.ConfigurationBuilder
import twitter4j.auth.OAuthAuthorization
import org.apache.spark.streaming.dstream._
import org.apache.spark.streaming.receiver.Receiver
import com.google.gson.Gson
import org.apache.spark.streaming.twitter.TwitterUtils
import twitter4j.Twitter
import twitter4j.TwitterFactory
import twitter4j.auth.AccessToken
import twitter4j.ResponseList
import twitter4j.QueryResult
import twitter4j.Query
import twitter4j.Paging
import scala.sys.process._
import scala.collection.mutable.ArrayBuffer

In [2]:
%scala
import org.apache.spark.streaming.twitter.TwitterUtils
import org.apache.spark.streaming.StreamingContext
import java.util.Properties
import org.apache.spark.streaming.twitter.TwitterInputDStream



val consumerKey = "test"
val consumerSecret = "test"
val accessToken = "test"
val accessTokenSecret = "test"


System.setProperty("twitter4j.oauth.consumerKey", consumerKey)
System.setProperty("twitter4j.oauth.consumerSecret", consumerSecret)
System.setProperty("twitter4j.oauth.accessToken", accessToken)
System.setProperty("twitter4j.oauth.accessTokenSecret", accessTokenSecret)


println("")
println("Create stream")
println("=============")
println("")
val filter = Array("test")
val ssc = new StreamingContext(sc, Seconds(30))
val stream = TwitterUtils.createStream(ssc, None, filter)
println("")
println("Extract texts and print")
println("=======================")
println("")


val statuses = stream.map(status => status.getText())
statuses.print()

var tweets = new ArrayBuffer[String]();
statuses.foreachRDD {
    tweets ++= _.collect() //you can now put it in an array or d w/e you want with it
}

println(" ")
println("Start")
println("=====")
println("")
ssc.start()
//ssc.awaitTermination()
ssc.awaitTerminationOrTimeout(180000) 


In [3]:
%scala
val tweets_text = tweets.mkString("|\n\n")
println(tweets_text)


In [4]:
%scala
dbutils.fs.rm("file:/dbfs/tmp/test")
// Write a file using the local file I/O API (over the fuse mount).
dbutils.fs.put("file:/dbfs/tmp/test", tweets_text)

// Unless you call this, the code below might not see the file or its latest contents.
"sync /dbfs/tmp/test" !

// Read the file using "dbfs:/" instead of the fuse mount.
dbutils.fs.head("dbfs:/tmp/test")